In [1]:
import os
from dotenv import load_dotenv
from IPython.display import display, Image

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.tools.retriever import create_retriever_tool
from langgraph.checkpoint.memory import InMemorySaver

from typing import List
import logging
from langchain_community.tools import BraveSearch
from langchain_tavily import TavilyExtract
import json

# For QA-Retriever
from langchain_core.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Custom Import
from optcg.vectorstore_logic import check_for_updates_to_rules, create_or_load_vectorstore_optcg_rulebooks
from optcg.tools import web_search_tool, youtube_search_tool, create_rulebook_retriever_tool, card_search_tool, get_board_tool
from optcg.agents import BaseAgent, RulebookAgent, ToollessAgent_4_1, ToollessAgent_o4_mini, SearchAgent_web_youtube, SearchAgent_web, SearchAgent_youtube, SearchAgent_web_youtube_rulebook, SearchAgent_web_rulebook, SearchAgent_youtube_rulebook
from optcg.routes import agent_routes, card_routes, board_routes


from langsmith import traceable
from langsmith.wrappers import wrap_openai
from openai import OpenAI
#openai_client = wrap_openai(OpenAI())

_ = load_dotenv() # Loads the .env file - e.g. the OPENAI_API_KEY

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)


In [7]:
agent = RulebookAgent()
agent.chat("what is the current board state?", verbose=True)

Loading existing vector store...


2025-07-28 19:21:59,098 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-28 19:22:03,185 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'messages': [HumanMessage(content='what is the current board state?', additional_kwargs={}, response_metadata={}, id='2e83b428-8715-486b-a00c-6479eb3ea2f7'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1vh4hEzjtObV3Bws2Cw0djKV', 'function': {'arguments': '{}', 'name': 'get_board_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 270, 'total_tokens': 281, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-BySOM0KMqYUdNT67yszspbxzf19CP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='RulebookAgent', id='run--a6518656-d04e-46db-a712-d6c0e41c342a-0', tool_calls=[{'name': 'get_board_tool', 'args': {}, 'id': 'call_

In [6]:
get_board_tool.invoke("what is on the board?")

{'UserState': {'life': 0,
  'don': 0,
  'leader': {'id': 'ST13-003_p1',
   'code': 'ST13-003',
   'rarity': 'L',
   'type': 'LEADER',
   'name': 'Monkey.D.Luffy',
   'images': {'small': 'https://en.onepiece-cardgame.com/images/cardlist/card/ST13-003_p1.png?241220',
    'large': 'https://en.onepiece-cardgame.com/images/cardlist/card/ST13-003_p1.png?241220'},
   'cost': 4,
   'attribute': {'name': 'Strike',
    'image': 'https://en.onepiece-cardgame.com/images/cardlist/attribute/ico_type01.png'},
   'power': 5000,
   'counter': '-',
   'color': 'Black/Yellow',
   'family': 'Supernovas/Straw Hat Crew',
   'ability': 'Your face-up Life cards are placed at the bottom of your deck instead of being added to your hand, according to the rules.<br>[DON!! x2] [Activate: Main] [Once Per Turn] You may trash 1 card from your hand: If you have 0 Life cards, add up to 2 Character cards with a cost of 5 from your hand or trash to the top of your Life cards face-up.',
   'trigger': '',
   'set': {'name'

In [18]:
import requests
api_base_url = os.getenv("API_BASE_URL", "http://localhost:8000")
print(f"Using API base URL: {api_base_url}")
print("{}/board/".format(api_base_url))
response = requests.get(
            f"{api_base_url}/board/"
        )
response.json()

Using API base URL: http://localhost:8000
http://localhost:8000/board/


{'UserState': {'life': 0,
  'don': 0,
  'leader': {'id': 'ST13-015_p1',
   'code': 'ST13-015',
   'rarity': 'SR',
   'type': 'CHARACTER',
   'name': 'Monkey.D.Luffy',
   'images': {'small': 'https://en.onepiece-cardgame.com/images/cardlist/card/ST13-015_p1.png?241220',
    'large': 'https://en.onepiece-cardgame.com/images/cardlist/card/ST13-015_p1.png?241220'},
   'cost': 5,
   'attribute': {'name': 'Strike',
    'image': 'https://en.onepiece-cardgame.com/images/cardlist/attribute/ico_type01.png'},
   'power': 6000,
   'counter': '1000',
   'color': 'Yellow',
   'family': 'Supernovas/Straw Hat Crew',
   'ability': '[Activate: Main] [Once Per Turn] This Character gains +2000 power until the start of your next turn. Then, if you have 1 or more Life cards, draw 1 card and trash 1 card from the top of your Life cards.',
   'trigger': '',
   'set': {'name': '-The Three Brothers-[ST13]'},
   'notes': []},
  'event': None,
  'stage': None,
  'character': []},
 'OpponentState': {'life': 0,
  '

In [ ]:
with open("test_state.json", "r") as f:
    game_state = json.load(f)
query = "What is the current game state? What are the next possible moves for each player?"
query = "What characters are on the board, and what are their abilities? What is the current life total for each player?"
query = "What should should I be wary of in the current game state?"

query_prompt_template = PromptTemplate.from_template("""
Here is the Current Game State:
{game_state} \n
You are an expert in the game of One Piece TCG. Based on the current game state, please address the following query. Try to keep the response concise and focused on the most relevant information for the query.:
{query}
""")
query_prompt = query_prompt_template.invoke({
    "game_state": game_state,
    "query": query
}).text

agent.chat(query_prompt, thread_id="test1234")

2025-07-24 16:35:11,572 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Here are the key things you should be wary of in the current game state:\n\n1. Opponent’s Leader Ability (Eustass "Captain" Kid):  \nKid can restand himself once per turn by resting 3 DON!! and trashing a card. This means your opponent can attack twice with their Leader, potentially putting more pressure on your Life or Characters.\n\n2. Opponent’s Board (Triple Nami):  \nYour opponent has three Nami (OP02-036), each with 5000 power. While their ability is mainly for searching {FILM} cards, they can still attack and pressure your Life or Characters. Be cautious if your Life is low or if you have vulnerable Characters.\n\n3. Opponent’s DON!! Advantage:  \nYour opponent has 8 DON!! available, giving them flexibility to power up attacks, use abilities, or play Events. Watch for potential power boosts or defensive Events during your attacks.\n\n4. Your Board State:  \nYou have strong attackers (Zoro Leader, Zoro Character with Rush, Nico Robin, Marco). However, be mindful of overextending

: 

'Here are the main things you should be wary of in the current game state:\n\n1. **Eustass "Captain" Kid Leader Ability**: Your opponent\'s leader can, once per turn, rest 3 DON!! and trash a card from their hand to set their Leader as active. This means Kid can attack twice in a turn, or attack and then be ready to block or threaten another attack. Be careful not to leave yourself open to multiple attacks, especially if you are low on life.\n\n2. **Multiple Nami (OP02-036) Characters**: Your opponent has three Nami (OP02-036) on the field. Each Nami can, when attacking (by resting 1 DON!!), look at the top 3 cards of their deck and add a {FILM} card (other than Nami) to their hand. This can quickly refill their hand and give them access to powerful {FILM} cards, including events or characters that could swing the game.\n\n3. **Potential for Multiple Attacks**: With three Nami and Kid, your opponent can attack multiple times in a turn. Even though Nami\'s power is only 5000, if you are not careful, you could lose several life in one turn, especially if you cannot block or counter enough attacks.\n\n4. **Trigger Effects**: When you take damage, remember that your opponent may be trying to push you into activating a [Trigger] effect from your life cards. Be aware of what triggers you have in your deck and how they might affect the game state.\n\n5. **Hand Size and Resources**: With Nami\'s effect, your opponent can increase their hand size and find key cards. If you let them keep all three Nami on the field, they will have a significant resource advantage over time.\n\n**Strategic Advice:**\n- Try to remove at least one or more Nami if possible, to slow down your opponent\'s card advantage.\n- Be cautious about overextending, as Kid\'s double attack can quickly deplete your life.\n- Plan your counters and blockers carefully, especially if you expect a big turn from your opponent.\n- Watch your own hand size and resources, as you may need to defend against multiple attacks in a single turn.\n\nIf you have any removal or ways to disrupt your opponent\'s board, now is a good time to use them!'


In [4]:
agent = RulebookAgent()
agent.chat(query_prompt, thread_id="test")
agent.chat("How do you recommend that I attack the opponet?", verbose=True, thread_id="test")

Loading existing vector store...


2025-07-24 16:13:46,721 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:13:47,972 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-24 16:13:49,098 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-24 16:14:03,002 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'messages': [HumanMessage(content='\nHere is the Current Game State:\n{\'UserState\': {\'life\': 4, \'don\': 7, \'leader\': {\'id\': \'OP01-001\', \'code\': \'OP01-001\', \'rarity\': \'L\', \'type\': \'LEADER\', \'name\': \'Roronoa Zoro\', \'images\': {\'small\': \'https://en.onepiece-cardgame.com/images/cardlist/card/OP01-001.png\', \'large\': \'https://en.onepiece-cardgame.com/images/cardlist/card/OP01-001.png\'}, \'cost\': 5, \'attribute\': {\'name\': \'Slash\', \'image\': \'https://en.onepiece-cardgame.com/images/cardlist/attribute/ico_type02.png\'}, \'power\': 5000, \'counter\': \'-\', \'color\': \'Red\', \'family\': \'Supernovas/Straw Hat Crew\', \'ability\': \'[DON!! x1] [Your Turn] All of your Characters gain +1000 power.\', \'trigger\': \'\', \'set\': {\'name\': \'-ROMANCE DAWN- [OP01]\'}, \'notes\': []}, \'event\': None, \'stage\': None, \'character\': [{\'id\': \'OP01-025\', \'code\': \'OP01-025\', \'rarity\': \'SR\', \'type\': \'CHARACTER\', \'name\': \'Roronoa Zoro\', \'i

In [4]:
class CardDBToolAgent(BaseAgent):
    # If model and temperature are not specified, defaults will be used, defined in BaseAgent.
    def __init__(self, model_name="gpt-4.1"):
        self.name = "CardDBToolAgent"
        super().__init__(model_name=model_name, temperature=0)
    
    def _create_prompt(self):
        """Create a system prompt for the agent. This should be tailored to the specific agent's purpose."""
        return "You are a helpful assistant that helps people answer information about the One Piece TCG. If you don't know the answer, just say you don't know. Do not try to make up an answer."

    def _setup_tools(self):
        """Setup tools for this agent. This can include any tools you want to use, or be an empty list if no tools are needed."""
        return [
            card_search_tool
        ]
agent = CardDBToolAgent(model_name="gpt-4.1")
agent.chat("What is a luffy card with power 2000?", verbose=True)

2025-07-08 10:48:14,168 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-08 10:48:23,571 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'messages': [HumanMessage(content='What is a luffy card with power 2000?', additional_kwargs={}, response_metadata={}, id='f0634f71-2b2f-4773-9a26-2f5360ba79ef'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qql4dwe1156hSAxtHpxKVV0k', 'function': {'arguments': '{"input":{"query":"Luffy","power":2000}}', 'name': 'card_search_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 227, 'total_tokens': 250, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-Br4qDTNS62PBFT25NEzWWTErLsRvw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='CardDBToolAgent', id='run--4c53ad13-a875-43d3-87f3-243e226e6315-0', tool_calls=[{'na